In [143]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from bs4 import BeautifulSoup
from nltk import word_tokenize as wt
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BVILLAMIL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BVILLAMIL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [144]:
df = pd.read_csv('Twitter.csv')
df

,clean_text,category,category_sentiment
0,i am not happy,-1,negative
1,i am not sad,1,positive
2,i'm fine,0,neutral
3,when modi promised “minimum government maximum...,-1,negative
4,talk all the nonsense and continue all the dra...,0,neutral
...,...,...,...
177971,'I'm not satisfied with The Hills finale. gon...,-1,negative
177972,this sucks,-1,negative
177973,this is bad,-1,negative
177974,I am not okay with this,-1,negative


In [145]:
df = df.dropna()
df = df.reset_index(drop=True)
df['category'].value_counts()

 1    72250
 0    62712
-1    43014
Name: category, dtype: int64

In [146]:
df = df.sample(frac=1).reset_index(drop=True)

#Using Twitter Dataset
data = df[df['category']==1][:43013]
data = data.append(df[df['category']==-1][:43013])
data = data.append(df[df['category']==0][:43013])


#Using Twitter_Data dataset
#data = df[df['category']==1][:2363]
#data = data.append(df[df['category']==-1][:2363])
#data = data.append(df[df['category']==0][:2363])

data = data.reset_index(drop=True)
display(data['category'].value_counts())

C:\Users\BVILLAMIL\AppData\Local\Temp\ipykernel_11096\1408359311.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df[df['category']==-1][:43013])
C:\Users\BVILLAMIL\AppData\Local\Temp\ipykernel_11096\1408359311.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df[df['category']==0][:43013])


 1    43013
-1    43013
 0    43013
Name: category, dtype: int64

In [147]:
data['pre_process'] = data['clean_text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['pre_process'] = data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
data['pre_process'] = data['pre_process'].apply(lambda x: re.sub(r"http\S+", "", x))

c:\Users\BVILLAMIL\Desktop\revalida2\env\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [148]:
#data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([stemmer.stem(w) for w in wt(x)]))
#data.head() 

In [149]:
def contractions(s):
    s = re.sub(r"won't", "will not", s)
    s = re.sub(r"wouldn't", "would not", s)
    s = re.sub(r"couldn't", "could not", s)
    s = re.sub(r"\'d", "would", s)
    s = re.sub(r"can\'t", "can not", s)
    s = re.sub(r"n\'t", "not", s)
    s = re.sub(r"\'re", "are", s)
    s = re.sub(r"\'s", "is", s)
    s = re.sub(r"\'ll", "will", s)
    s = re.sub(r"\'t", "not", s)
    s = re.sub(r"\'ve", "have", s)
    s = re.sub(r"\'m", "am", s)
    return s

data['pre_process'] = data['pre_process'].apply(lambda x:contractions(x))
data

,clean_text,category,category_sentiment,pre_process
0,what makes modi the best all time via,1,positive,what makes modi the best all time via
1,nirav refused bail the court judgelutyens pidi...,1,positive,nirav refused bail the court judgelutyens pidi...
2,modi was only contesting mla seats until then ...,1,positive,modi was only contesting mla seats until then ...
3,something your saying means others laugh youon...,1,positive,something your saying means others laugh youon...
4,sir old enough get some proper education not b...,1,positive,sir old enough get some proper education not b...
...,...,...,...,...
129034,modi should have dismissed the mamta govt for ...,0,neutral,modi should have dismissed the mamta govt for ...
129035,2014 was not modi wave was “modi media” wave,0,neutral,2014 was not modi wave was “modi media” wave
129036,this article from 2010 had acquired this capab...,0,neutral,this article from 2010 had acquired this capab...
129037,even governor’ are campaigning for chor modi s...,0,neutral,even governor’ are campaigning for chor modi s...


In [150]:
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in wt(x)]))
data

,clean_text,category,category_sentiment,pre_process
0,what makes modi the best all time via,1,positive,what makes modi the best all time via
1,nirav refused bail the court judgelutyens pidi...,1,positive,nirav refused bail the court judgelutyens pidi...
2,modi was only contesting mla seats until then ...,1,positive,modi was only contesting mla seats until then ...
3,something your saying means others laugh youon...,1,positive,something your saying means others laugh youon...
4,sir old enough get some proper education not b...,1,positive,sir old enough get some proper education not b...
...,...,...,...,...
129034,modi should have dismissed the mamta govt for ...,0,neutral,modi should have dismissed the mamta govt for ...
129035,2014 was not modi wave was “modi media” wave,0,neutral,was not modi wave was modi media wave
129036,this article from 2010 had acquired this capab...,0,neutral,this article from had acquired this capabilit...
129037,even governor’ are campaigning for chor modi s...,0,neutral,even governor are campaigning for chor modi s...


In [151]:
stop = stopwords.words('english')
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))
data

,clean_text,category,category_sentiment,pre_process
0,what makes modi the best all time via,1,positive,makes modi best time via
1,nirav refused bail the court judgelutyens pidi...,1,positive,nirav refused bail court judgelutyens pidis pl...
2,modi was only contesting mla seats until then ...,1,positive,modi contesting mla seats gujarat first time c...
3,something your saying means others laugh youon...,1,positive,something saying means others laugh youonly la...
4,sir old enough get some proper education not b...,1,positive,sir old enough get proper education believe wh...
...,...,...,...,...
129034,modi should have dismissed the mamta govt for ...,0,neutral,modi dismissed mamta govt obstruting central a...
129035,2014 was not modi wave was “modi media” wave,0,neutral,modi wave modi media wave
129036,this article from 2010 had acquired this capab...,0,neutral,article acquired capability way back modi crow...
129037,even governor’ are campaigning for chor modi s...,0,neutral,even governor campaigning chor modi shah bjp r...


In [152]:
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in wt(x)]))
data

,clean_text,category,category_sentiment,pre_process
0,what makes modi the best all time via,1,positive,make modi best time via
1,nirav refused bail the court judgelutyens pidi...,1,positive,nirav refused bail court judgelutyens pidis pl...
2,modi was only contesting mla seats until then ...,1,positive,modi contesting mla seat gujarat first time co...
3,something your saying means others laugh youon...,1,positive,something saying mean others laugh youonly lau...
4,sir old enough get some proper education not b...,1,positive,sir old enough get proper education believe wh...
...,...,...,...,...
129034,modi should have dismissed the mamta govt for ...,0,neutral,modi dismissed mamta govt obstruting central a...
129035,2014 was not modi wave was “modi media” wave,0,neutral,modi wave modi medium wave
129036,this article from 2010 had acquired this capab...,0,neutral,article acquired capability way back modi crow...
129037,even governor’ are campaigning for chor modi s...,0,neutral,even governor campaigning chor modi shah bjp r...


In [153]:
x = data['pre_process']
y = data['category_sentiment']
y_strat = data['category']

#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [154]:
x_vectorized

<129039x88631 sparse matrix of type '<class 'numpy.float64'>'
	with 1662506 stored elements in Compressed Sparse Row format>

In [155]:
#x_train, x_test, y_train, y_test = train_test_split(x_vectorized, y, test_size=0.25, random_state=30, stratify=y)
x_train, x_test, y_train, y_test = train_test_split(x_vectorized, y, test_size=0.2, random_state=42, stratify=y_strat)
print("Train:", x_train.shape, y_train.shape)
print("Test: ", x_test.shape, y_test.shape)

Train: (103231, 88631) (103231,)
Test:  (25808, 88631) (25808,)


In [156]:
regressor = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = regressor.fit(x_train, y_train)

In [157]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
gs_clf = GridSearchCV(model, params, n_jobs=1, cv=5)
gs_clf = gs_clf.fit(x_train, y_train)
model = gs_clf.best_estimator_

c:\Users\BVILLAMIL\Desktop\revalida2\env\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\Users\BVILLAMIL\Desktop\revalida2\env\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [158]:
y_pred = model.predict(x_test)

report = classification_report(y_test, y_pred, output_dict=True)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.85      0.81      0.83      8602
     neutral       0.83      0.92      0.87      8603
    positive       0.87      0.82      0.85      8603

    accuracy                           0.85     25808
   macro avg       0.85      0.85      0.85     25808
weighted avg       0.85      0.85      0.85     25808



Testing the Sentiment Analysis
-1 = Negative
 0 = Neutral
 1 = Positive

In [159]:
test_feature = vectorizer.transform(["i am happy"])
model.predict(test_feature)

array(['positive'], dtype=object)

In [160]:
pickl = {'vectorizer': vectorizer,
         'model': model
         }
pickle.dump(pickl, open('models2'+".p", "wb"))